# SFT

In [3]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load dataset
dataset = load_dataset("HuggingFaceTB/smoltalk", "all")

# Configure model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(
    device
)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Setup chat template
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Configure trainer
training_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
)

# Start training
trainer.train()

Tokenizing train dataset:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (8567 > 8192). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/1043917 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/54948 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/54948 [00:00<?, ? examples/s]

* Trackio project initialized: huggingface
* Trackio metrics logged to: /root/autodl-tmp/huggingface/trackio
* View dashboard by running in your terminal:
trackio show --project "huggingface"
* or by running in Python: trackio.show(project="huggingface")


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,1.602900,1.587764,1.612800,131512.000000,0.627473
100,1.549000,1.516506,1.529900,266413.000000,0.641325
150,1.306900,1.492533,1.475480,383164.000000,0.645330
200,1.535500,1.476815,1.501044,507210.000000,0.647983
250,1.503700,1.467190,1.490720,634005.000000,0.648918
300,1.451700,1.459018,1.479832,759575.000000,0.650634
350,1.448100,1.453230,1.445135,884080.000000,0.651683
400,1.423800,1.447176,1.472528,1007609.000000,0.652590
450,1.423700,1.441910,1.460203,1138891.000000,0.653445
500,1.471200,1.438264,1.455391,1273741.000000,0.654492


* Run finished. Uploading logs to Trackio Space: http://127.0.0.1:7860/ (please wait...)


TrainOutput(global_step=1000, training_loss=1.4609604654312134, metrics={'train_runtime': 7790.8407, 'train_samples_per_second': 0.513, 'train_steps_per_second': 0.128, 'total_flos': 2478923753836032.0, 'train_loss': 1.4609604654312134, 'epoch': 0.0038317112422407845})

In [2]:
# 上传模型到 Hugging Face Hub
from huggingface_hub import HfApi, create_repo
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import shutil

# 配置参数
repo_name = "smoltalk-sft-135M-chinese"  # 修改为你的仓库名称
checkpoint_path = "./sft_output/checkpoint-1000"  # 最新的检查点
final_model_path = "./sft_output/final_model"

# 从检查点加载模型和tokenizer
print("正在从检查点加载模型...")
try:
    # 如果之前已经训练过，从检查点加载
    if os.path.exists(checkpoint_path):
        model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
        tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
        print(f"✅ 已从 {checkpoint_path} 加载模型")
    else:
        print("❌ 检查点不存在，请先运行训练代码")
        raise FileNotFoundError(f"找不到检查点: {checkpoint_path}")
        
except NameError:
    # 如果 model 没有定义，尝试从检查点加载
    if os.path.exists(checkpoint_path):
        model = AutoModelForCausalLM.from_pretrained(checkpoint_path)
        tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
        print(f"✅ 已从 {checkpoint_path} 加载模型")
    else:
        print("❌ 请先运行训练代码生成模型")

# 保存最终模型和tokenizer
print(f"正在保存模型到 {final_model_path}...")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
print("✅ 模型已保存")

# 创建中文 README
readme_content = """---
language: zh
license: apache-2.0
tags:
- text-generation
- causal-lm
- sft
- chinese
datasets:
- HuggingFaceTB/smoltalk
base_model: HuggingFaceTB/SmolLM2-135M
widget:
- text: "你好，请问"
---

# SmolTalk SFT 135M 中文对话模型

## 模型描述

这是一个基于 SmolLM2-135M 模型使用 SFT (Supervised Fine-Tuning) 方法在 SmolTalk 数据集上微调的中文对话模型。

## 训练信息

- **基础模型**: HuggingFaceTB/SmolLM2-135M
- **训练数据集**: HuggingFaceTB/smoltalk
- **训练步数**: 1000 steps
- **批次大小**: 4
- **学习率**: 5e-5
- **训练框架**: TRL (Transformer Reinforcement Learning)

## 使用方法

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "your-username/smoltalk-sft-135M-chinese"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 生成对话
inputs = tokenizer("你好，请问", return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)
```

## 训练配置

- 最大步数: 1000
- 每设备训练批次大小: 4
- 学习率: 5e-5
- 日志记录步数: 10
- 保存步数: 100
- 评估策略: steps
- 评估步数: 50

## 性能指标

- 训练损失: ~1.46
- 训练时长: ~2小时
- 训练样本/秒: ~0.513

## 注意事项

- 该模型主要用于中文对话生成任务
- 模型参数量较小（135M），适合资源受限的场景
- 建议根据具体应用场景进一步微调

## 许可证

Apache 2.0

## 引用

如果你使用了这个模型，请引用原始的 SmolLM2 和 SmolTalk 项目。
"""

# 保存 README
with open(os.path.join(final_model_path, "README.md"), "w", encoding="utf-8") as f:
    f.write(readme_content)

print("✅ README.md 已创建")

# 删除多余的检查点（只保留最后一个）
print("\n开始清理旧的检查点...")
checkpoints_to_delete = [
    "./sft_output/checkpoint-100",
    "./sft_output/checkpoint-200",
    "./sft_output/checkpoint-300",
    "./sft_output/checkpoint-400",
    "./sft_output/checkpoint-500",
    "./sft_output/checkpoint-600",
    "./sft_output/checkpoint-700",
    "./sft_output/checkpoint-800",
    "./sft_output/checkpoint-900",
]

deleted_count = 0
for checkpoint in checkpoints_to_delete:
    if os.path.exists(checkpoint):
        shutil.rmtree(checkpoint)
        print(f"  ✅ 已删除: {checkpoint}")
        deleted_count += 1

if deleted_count > 0:
    print(f"\n✅ 清理完成！删除了 {deleted_count} 个检查点，只保留了 checkpoint-1000")
else:
    print("\n✅ 没有需要清理的检查点")
    
print(f"\n✅ 所有任务完成！模型已保存到: {final_model_path}")

/root/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


正在从检查点加载模型...
✅ 已从 ./sft_output/checkpoint-1000 加载模型
正在保存模型到 ./sft_output/final_model...
✅ 模型已保存
✅ README.md 已创建

开始清理旧的检查点...
  ✅ 已删除: ./sft_output/checkpoint-100
  ✅ 已删除: ./sft_output/checkpoint-200
  ✅ 已删除: ./sft_output/checkpoint-300
  ✅ 已删除: ./sft_output/checkpoint-400
  ✅ 已删除: ./sft_output/checkpoint-500
  ✅ 已删除: ./sft_output/checkpoint-600
  ✅ 已删除: ./sft_output/checkpoint-700
  ✅ 已删除: ./sft_output/checkpoint-800
  ✅ 已删除: ./sft_output/checkpoint-900

✅ 清理完成！删除了 9 个检查点，只保留了 checkpoint-1000

✅ 所有任务完成！模型已保存到: ./sft_output/final_model


In [ ]:
# 上传到 Hugging Face Hub
# 注意：需要先登录或设置 token
from huggingface_hub import HfApi, login

# 方法2：直接设置 token
hub_token = "xxx"
api = HfApi(token=hub_token)

# 上传模型（需要先取消上面某个方法的注释并设置 token）
def upload_model_to_hub(repo_name, local_path="./sft_output/final_model", private=False):
    """
    上传模型到 Hugging Face Hub
    
    Args:
        repo_name: 仓库名称，格式为 "username/model-name"
        local_path: 本地模型路径
        private: 是否设置为私有仓库
    """
    try:
        # 创建仓库
        api = HfApi()
        repo_url = api.create_repo(
            repo_id=repo_name,
            repo_type="model",
            private=private,
            exist_ok=True
        )
        print(f"✅ 仓库已创建/存在: {repo_url}")
        
        # 上传整个文件夹
        api.upload_folder(
            folder_path=local_path,
            repo_id=repo_name,
            repo_type="model",
        )
        
        print(f"✅ 模型已成功上传到: https://huggingface.co/{repo_name}")
        return repo_url
        
    except Exception as e:
        print(f"❌ 上传失败: {e}")
        print("请确保：")
        print("1. 已经登录 Hugging Face (使用 login() 或设置 token)")
        print("2. repo_name 格式正确 (username/model-name)")
        print("3. 有足够的权限创建仓库")
        return None

# 使用示例（取消注释并修改参数后运行）
repo_name = "yiwenX/smoltalk-sft-135M-chinese"  # 修改为你的用户名和模型名
upload_model_to_hub(repo_name, private=False)

✅ 仓库已创建/存在: https://huggingface.co/yiwenX/smoltalk-sft-135M-chinese


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...utput/final_model/model.safetensors:   0%|          | 12.0kB /  538MB            

✅ 模型已成功上传到: https://huggingface.co/yiwenX/smoltalk-sft-135M-chinese


RepoUrl('https://huggingface.co/yiwenX/smoltalk-sft-135M-chinese', endpoint='https://huggingface.co', repo_type='model', repo_id='yiwenX/smoltalk-sft-135M-chinese')

In [2]:
# 测试最终模型
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 设置设备
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"使用设备: {device}")

# 从 final_model 文件夹加载模型
model_path = "./sft_output/final_model"
print(f"正在从 {model_path} 加载模型...")

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 将模型移到设备
model = model.to(device)
model.eval()  # 设置为评估模式
print("✅ 模型加载成功!")

# 测试函数
def generate_response(prompt, max_length=100, temperature=0.7, top_p=0.9, do_sample=True):
    """
    生成对话回复
    
    Args:
        prompt: 输入提示
        max_length: 最大生成长度
        temperature: 控制生成的随机性（0-1，越高越随机）
        top_p: nucleus sampling 参数
        do_sample: 是否采样（False则使用greedy decoding）
    """
    # 准备输入
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 生成
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # 解码输出
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 测试示例
print("\n" + "="*50)
print("开始测试模型...")
print("="*50)

# 测试用例列表 - 3个简单英文问题
test_prompts = [
    "What is your name?",
    "How are you today?",
    "What can you do?",
]

# 对每个测试用例生成回复
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[测试 {i}]")
    print(f"输入: {prompt}")
    response = generate_response(prompt, max_length=100, temperature=0.7)
    print(f"输出: {response}")
    print("-" * 40)

使用设备: cuda
正在从 ./sft_output/final_model 加载模型...
✅ 模型加载成功!

开始测试模型...

[测试 1]
输入: What is your name?
输出: What is your name? Please provide a simple, yet concise response."

"Hi Emily," he said, his voice firm but kind. "I'm Alex. I'm a software engineer at a startup. I've been working on this project for a while now and I wanted to share my thoughts."

Alex was a bit hesitant at first, but he was determined to prove that he was a valuable member of the team. He was an introvert, but he had a knack
----------------------------------------

[测试 2]
输入: How are you today?
输出: How are you today?

What is the first thing that comes to mind when you think of today?

1. What are some of the most memorable experiences you’ve had?

2. What are some of the most exciting or interesting experiences you’ve had?

3. What is the most memorable thing that happened to you in the last week?

4. What are some of the most challenging experiences you’ve had?

5. What is your best memory of this
-------------

In [3]:
# 对比测试：微调前的原始模型
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

print("="*50)
print("加载原始模型进行对比...")
print("="*50)

# 设置设备
device = "cuda" if torch.cuda.is_available() else "cpu"

# 加载原始的预训练模型（微调前）
original_model_name = "HuggingFaceTB/SmolLM2-135M"
print(f"正在加载原始模型: {original_model_name}")

original_model = AutoModelForCausalLM.from_pretrained(original_model_name).to(device)
original_tokenizer = AutoTokenizer.from_pretrained(original_model_name)
original_model.eval()

print("✅ 原始模型加载成功!")

# 使用相同的生成函数
def generate_original_response(prompt, max_length=100, temperature=0.7, top_p=0.9, do_sample=True):
    inputs = original_tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = original_model.generate(
            **inputs,
            max_length=max_length,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            pad_token_id=original_tokenizer.eos_token_id if original_tokenizer.eos_token_id else original_tokenizer.pad_token_id,
            eos_token_id=original_tokenizer.eos_token_id if original_tokenizer.eos_token_id else original_tokenizer.pad_token_id
        )
    
    response = original_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 对比测试相同的问题
print("\n" + "="*50)
print("原始模型测试结果（微调前）")
print("="*50)

test_prompts = [
    "What is your name?",
    "How are you today?",
    "What can you do?",
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n[测试 {i}]")
    print(f"输入: {prompt}")
    response = generate_original_response(prompt, max_length=100, temperature=0.7)
    print(f"原始模型输出: {response}")
    print("-" * 40)

print("\n" + "="*50)
print("对比总结：")
print("- 原始模型：基础的语言生成能力")
print("- 微调模型：经过 SFT 训练后的对话能力")
print("- 可以运行上面两个 cell 对比效果差异")
print("="*50)

加载原始模型进行对比...
正在加载原始模型: HuggingFaceTB/SmolLM2-135M
✅ 原始模型加载成功!

原始模型测试结果（微调前）

[测试 1]
输入: What is your name?
原始模型输出: What is your name?


What is your birthday?


What is your favorite color?


What is your favorite food?


What is your favorite type of music?


What is your favorite sport?


What is your favorite food?


What is your favorite animal?


What is your favorite book?


What is your favorite movie?


What is your favorite TV show?


What is your favorite book?


What is your
----------------------------------------

[测试 2]
输入: How are you today?
原始模型输出: How are you today?

It is a question that I have asked myself for months, and I have never been able to get to the answer.

I am trying to answer this question in this blog post, because I am not sure how to do so.

In this blog post, I will try to answer this question in a way that is easy for you to understand.

It is not a question that I would answer in a simple way, because it is not
-----------------------------------